In [ ]:
import numpy as np
import pandas as pd
class tableu:
  def __init__(self,minmax):
        self.obj = []
        self.row = []
        self.Rhs = []
        self.slack =[]
        self.Basic = []
        self.coeficient = []
        self.minmax = minmax
        self.table = np.empty_like
        self.bfs = True

  def add_objective(self, objective_function):
        self.obj += objective_function
        for j in range(len(self.obj)):
         self.obj[j] *= -1

  def add_constraint(self,constraint,cons,based_on_sign):   #add slack variables too
    self.row.append(constraint)
    self.Rhs.append(cons)
    self.slack.append(based_on_sign)

  def setObj(self,minmax="MAX"):
    if(minmax == "MIN" or minmax == "MAX"):
      self.minmax = minmax

  def number_vars(self):
    no_s_vars= self.slack.count(1)+self.slack.count(-1)
    no_art_vars=self.slack.count(0)+self.slack.count(-1)
    return no_s_vars,no_art_vars

  def setcoeificient(self):
   no_s_vars =self.number_vars()[0]
   no_art_vars =self.number_vars()[1]
   for i in range(1,len(self.obj)+1):
      self.coeficient.append('x_'+str(i))
   if no_s_vars != 0:
    for k in range(1,no_s_vars+1):
     self.coeficient.append('s_'+str(k))
   if no_art_vars != 0:
    for m in range(1,no_art_vars+1):
     self. coeficient.append('r_'+str(m))
   print(self.coeficient)



  def getfirsttable(self):
    m =len(self.row)+1
    z = np.zeros((m,len(self.row)),dtype=float,order='c')
    for i in range(1,m):
      z[i][i-1] = self.slack[i-1]
    ar_row =[]
    for i in range(len(self.slack)):
      if self.slack[i]!= 1:
        ar_row.append(i)
    new_ar_row =ar_row.copy()
    print(ar_row)
  #construction

    table = [self.obj] + self.row
    self.table = np.array(table, dtype=float)
    self.table =np.hstack((self.table,z))
  # w setup
    for i in range(len(self.obj),len(self.obj)+len(self.row)):
        if np.sum(self.table[:,i]) != 1:
          self.bfs = False
          art_var = np.zeros((len(self.row)+1,1),dtype=int)
          j = new_ar_row.pop(0)+1
          art_var[j,0] = 1
          art_var[0,0] = -1
          self.table = np.hstack((self.table,art_var))

    self.Rhs =[0]+self.Rhs
    self.Rhs = np.array(self.Rhs)
    self.table = np.hstack((self.table, self.Rhs[:,np.newaxis]))


    if self.bfs == False:
      for i in range(len(self.obj)):
        self.table[0,i]=0
    index=[]
    for i in range(len(self.obj),len(self.obj)+len(self.row)):
        if np.sum(self.table[:,i])==0:
          index.append(i)
    self.table=np.delete(self.table,index,axis=1)
    for x in ar_row:
        self.table[0] += self.table[x+1]

    self.Basic=([0]*len(self.row))
    k =1
    for i in ar_row:
      self.Basic[i]='r_'+str(k)
      k += 1
    t = 1 + self.slack.count(-1)
    for i in range(len(self.Basic)):
        if self.Basic[i] == 0:
           self.Basic[i]='s_'+str(t)
           t += 1
    self.table=pd.DataFrame(self.table)
    if self.bfs ==False:
      self.table.index = ['w']+self.Basic
    else:
      self.table.index=['z']+self.Basic
    self.table.columns=self.coeficient+['Rhs']

    print(self.table)

  def display(self):
    for i in range(self.table.shape[0]):
      for j in range(self.table.shape[1]):
         print(round(self.table[i,j],3))
    print()
  def optimality_check(self):
    optimal =False
    if self.minmax== "MAX":
      if min((self.table.values[0,:-1]))< 0:
          optimal= False
      else:
          optimal = True
    else:
      if max((self.table.values[0,:-1]))> 0:
           optimal=False
      else:
           optimal =True
    return optimal

  def pivot_coloumn(self):
    if self.bfs==True:
      if self.minmax == "MAX":
          column = self.table.values[1:,np.argmin(self.table.values[0,:-1])]
      else:
          column= self.table.values[1:,np.argmax(self.table.values[0,:-1])]
    else:
      column=column= self.table.values[1:,np.argmax(self.table.values[0,:-1])]

    is_bounded = 0

    if np.where(column>0)[0].size==0:
          is_bounded=-1
    else:
        is_bounded = 1

    return column,is_bounded

  def pivot_row (self) :
    column=self.pivot_coloumn()[0]

    rhs =np.array(self.table.values[1:,-1])
    initiate=rhs/column

    rhs= np.delete(rhs, np.where(column<0))
    column = np.delete(column, np.where(column<0)) #delete negative elements in column
    if self.minmax=="MAX":
      entry =np.argmin(self.table.values[0,:-1])
    else:
      entry=np.argmax(self.table.values[0,:-1])
    arr = (rhs/column)
    ratio = arr.tolist()
    ratio.sort()
    if ratio.count(ratio[0])>1:
      result= np.where(initiate==min(arr))
      print(result)
      print("degeneracy alarm")
      out=result[0][0]+1 #or result[0][1]and so on
    else:
      initiate=initiate.tolist()
      out =initiate.index(min(arr))+1
    return out,entry

  def pivot_calculate(self,out,entry):
    self.table.values[out,:] = self.table.values[out,:]/self.table.values[out,entry]
    for i in range(self.table.values.shape[0]):
      if i != out:
        self.table.values[i,:] = self.table.values[i,:]-self.table.values[out,:]*(self.table.values[i,entry])/self.table.values[out,entry]
  def simplex(self):
    optimal=False
    iter = 1
    while True:
      c =self.pivot_coloumn()[0]
      bounded=self.pivot_coloumn()[1]
      if bounded == -1:
        print("problem is unbounded")
        break
      r=self.pivot_row()[0] #return out
      print(r)
      input=self.pivot_row()[1]
      self.pivot_calculate(r,input)

      #new basic variables
      self.Basic[r-1] =self.coeficient[input]
      self.table.index=['Z']+self.Basic
      self.table.columns = self.coeficient + ['Rhs']
      iter +=1
      print("----------------------------------")
      print("Iteration :", iter)
      print(round(self.table,3))
      rhs =np.array(self.table.values[1:,-1])
      degeneracy=np.where(rhs==0)
      if degeneracy[0].size!=0:
        print('it has been degenerated')
      for i in range(self.table.values.shape[1]-1):
        if (self.table.values[0,i]==0 and self.coeficient[i] not in self.Basic):
          print("multiple optimal solution")

      optimal = self.optimality_check()
      if optimal==True:
        print("solved")
        break
  def drive_out_ar(self):  #phase1
    if self.bfs==False:
      no_s_vars =self.number_vars()[0]
      no_art_vars =self.number_vars()[1]
      iter=1
    while True:
        n =self.table.values[1:,np.argmax(self.table.values[0,:-1])]
        r = self.pivot_row()[0] #outrow
        print(r)
        input=np.argmax(self.table.values[0,:-1])
        self.pivot_calculate(r,input)
        self.Basic[r-1] =self.coeficient[input]

        self.table.index=['w']+self.Basic
        self.table.columns=self.coeficient + ['Rhs']
        iter +=1
        print("----------------------------------")
        print("iteration",iter)
        print(round(self.table,2))
        print(self.bfs)
        for j in range(1,(no_art_vars)+1):
          if ("r_"+str(j))  in self.Basic:
            break
        else:

          if self.table.values[0,-1]>=-0.01:
            self.bfs = True
            print("bfs exist")
            break
        if self.table.values[0,-1]<-0.01:
            print("infeaseable solution")
            break
        if np.where(self.table.values[0,:-1]>0.001)[0].size==0:
          break
  def phaese_2(self):
    print('phase2')
    no_s_vars=self.number_vars()[0]
    m = len(self.obj)+no_s_vars

    if self.bfs == True:
      self.table.index=['z']+self.Basic
      self.table.drop(self.table.iloc[:, m:-1], inplace = True, axis = 1)

      self.table.values[0,:-1]=self.obj+([0] *no_s_vars)
      print(round(self.table,2))
      #prepration
      for i,j in enumerate(self.Basic):
        col=self.coeficient.index(j)
        self.table.values[0] -= self.table.values[0][col] * self.table.values[i+1]
      print("----------------------------------")
      print(self.table)
      self.coeficient=np.delete(self.coeficient,np.s_[m:])
      self.coeficient=self.coeficient.tolist()

      self.simplex()












In [ ]:
model1 =tableu("MAX")
model1.add_objective([3,2,5])
model1.add_constraint([1,2,1],430,1)
model1.add_constraint([3,0,2],460,1)
model1.add_constraint([1,4,0],420,1)
model1.setcoeificient()
model1.getfirsttable()
model1.simplex()
#model1.drive_out_ar()
#model1.phaese_2()











['x_1', 'x_2', 'x_3', 's_1', 's_2', 's_3']
[]
     x_1  x_2  x_3  s_1  s_2  s_3    Rhs
z   -3.0 -2.0 -5.0  0.0  0.0  0.0    0.0
s_1  1.0  2.0  1.0  1.0  0.0  0.0  430.0
s_2  3.0  0.0  2.0  0.0  1.0  0.0  460.0
s_3  1.0  4.0  0.0  0.0  0.0  1.0  420.0
2
----------------------------------
Iteration : 2
     x_1  x_2  x_3  s_1  s_2  s_3     Rhs
Z    4.5 -2.0  0.0  0.0  2.5  0.0  1150.0
s_1 -0.5  2.0  0.0  1.0 -0.5  0.0   200.0
x_3  1.5  0.0  1.0  0.0  0.5  0.0   230.0
s_3  1.0  4.0  0.0  0.0  0.0  1.0   420.0
1
----------------------------------
Iteration : 3
      x_1  x_2  x_3  s_1   s_2  s_3     Rhs
Z    4.00  0.0  0.0  1.0  2.00  0.0  1350.0
x_2 -0.25  1.0  0.0  0.5 -0.25  0.0   100.0
x_3  1.50  0.0  1.0  0.0  0.50  0.0   230.0
s_3  2.00  0.0  0.0 -2.0  1.00  1.0    20.0
solved


<ipython-input-3-c3ba72696c0c>:151: RuntimeWarning: divide by zero encountered in true_divide
  initiate=rhs/column
<ipython-input-3-c3ba72696c0c>:159: RuntimeWarning: divide by zero encountered in true_divide
  arr = (rhs/column)
<ipython-input-3-c3ba72696c0c>:151: RuntimeWarning: divide by zero encountered in true_divide
  initiate=rhs/column
<ipython-input-3-c3ba72696c0c>:159: RuntimeWarning: divide by zero encountered in true_divide
  arr = (rhs/column)


In [ ]:
class Person:
  def __init__(self, minmax):
    self.minmax = None
    self.basic =[]
  def setObj(self):
    if(self.minmax == "MIN" or self.minmax == "MAX"):
      print(self.minmax)
      return self.minmax
  def printname(self):
    if(self.minmax=="Min"or"Max"):
      self.minmax ="Min"
      b = self.setObj()
      print(self.minmax)
      print(b)
    self.basic=[1,2,3]
    print(self.basic)
  def acs(self,e,g):
    self.basic+=[6,4,1]
    a=[e]
    b=[g]
    self.basic+=a +b
    print(self.basic)





#Use the Person class to create an object, and then execute the printname method:

x = Person( "MAX")
x.setObj()
x.printname()
x.acs(1,2)

Min
None
[1, 2, 3]
[1, 2, 3, 6, 4, 1, 1, 2]
